In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 21807938
paper_name = 'guerra_frisan_2011' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/Genes_identified_yeast_screen.xlsx', sheet_name='Sheet1')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 13 x 5


In [7]:
original_data.head()

,Group,Function,# genes,Yeast genesa,Human orthologs
0,1.0,Replication recombination repair,13.0,"DCC1, DDC1, DIA2, ELG1, MMS4, MUS81, RAD27, RA...","DSCC1, DIAPH3, MUS81, FEN1, RAD50, RAD51, XRCC..."
1,2.0,Checkpoint,5.0,"CHK1, MEC3, MRC1, RAD9, TOF1","CHEK1, PIP3-E, MRC1L.1, RAD9A,"
2,3.0,Chromatin silencing telomeres,8.0,"CST6, ESC1, NAM7, NAT3, NMD2, NPT1, UPF3, YPL205C","CST6, UPF1, NAT5, UPF2, SLC17A1, UPF3A"
3,4.0,Mitotic chromosome transmission,5.0,"CHL1, CHL4, CTF4, MCM21, MCM22","DDX11, CENPN, REPS2, NFIC"
4,5.0,Vacuolar/ Golgi/ endocytosis,10.0,"DRS2, PEP7, PEP12, PMR1, SNF7, VPS3, VPS4, VPS...","ATP8A2, LAP3, STX12, ATP2C1, CHMP4B, VPS4B, VP..."


In [8]:
original_data['genes'] = original_data['Yeast genesa'].apply(lambda x: x.split(','))

In [9]:
all_genes = [gene for lst in original_data['genes'] for gene in lst]

In [11]:
original_data = pd.DataFrame(data={'gene': all_genes})

In [12]:
original_data['gene'] = original_data['gene'].astype(str)

In [13]:
# Eliminate all white spaces & capitalize
original_data['gene'] = clean_genename(original_data['gene'])

In [14]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['gene'], to='orf')

In [15]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

                gene      orf
index_input                  
78           78GENES  78GENES


In [16]:
original_data = original_data.loc[t,:]

In [17]:
original_data['data'] = -1

In [18]:
original_data.set_index('orf', inplace=True)

In [19]:
original_data = original_data[['data']].copy()

In [20]:
original_data = original_data.groupby(original_data.index).mean()

In [21]:
original_data.shape

(78, 1)

# Prepare the final dataset

In [22]:
data = original_data.copy()

In [23]:
dataset_ids = [16699]
datasets = datasets.reindex(index=dataset_ids)

In [24]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [25]:
data.head()

dataset_id,16699
data_type,value
orf,
YAL002W,-1
YAL016W,-1
YAL026C,-1
YBL058W,-1
YBR098W,-1


## Subset to the genes currently in SGD

In [26]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [27]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,16699
,data_type,value
gene_id,orf,
2,YAL002W,-1
14,YAL016W,-1
24,YAL026C,-1
146,YBL058W,-1
293,YBR098W,-1


# Normalize

In [28]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [29]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [30]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id      16699         
data_type       value   valuez
gene_id orf                   
2       YAL002W    -1 -7.64266
14      YAL016W    -1 -7.64266
24      YAL026C    -1 -7.64266
146     YBL058W    -1 -7.64266
293     YBR098W    -1 -7.64266

# Print out

In [31]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [32]:
from IO.save_data_to_db3 import *

In [33]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 21807938...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00,  3.71it/s]

Updating the data_modified_on field...
